In [1]:
import pandas as pd
import numpy as np
import time
import datetime
from statsmodels.nonparametric.kernel_density import KDEMultivariate
import matplotlib.pyplot as plt
from sklearn.neighbors import DistanceMetric
from sklearn.neighbors import NearestNeighbors

In [ ]:
def entrenar(siedco, convergencia = 0.05, dist_max = 0.3, dist_t_max=90):
    """Funcion para entrenar el modelo de prediccion de crimen de Bogota como un proceso puntual autoexcitado
     Metodología basada en Mohler et al. (2012) Self-Exciting Point Process Modeling of Crime 
    
     localidad: Localidad en que se entrena el modelo. Tiene en cuenta crímenes ocurridos a una dist_max de la frontera de la localidad.
     fecha_inicial: Fecha de inicio de los datos de entrenamiento
     fecha_final: Fecha final de los datos de entrenamiento
     convergencia: Parametro de convergencia del modelo
     dist_max: Distancia espacial maxima para ser considerado un crimen replica de otro
     dist_t_max: Distancia temporal maxima para ser considerado un crimen replica de otro

     Retorna:
     Kernels que capturan patrones espacio-temporales y de replica de crimenes, datos de entrenamiento utilizados, codigo de la localidad, tiempo de entrenamiento, y nivel de convergencia alcanzado 

    """
    k_g = 80
    k_mu = 15
    bw_nu = 0.01   
    
    #siedco = datos_procesados(localidad, fecha_inicial, fecha_final, dist_max)

    restas = matriz_distancias(siedco[['LONGITUD_X', 'LATITUD_Y', 'turno_lineal']])

    inicio = time.time()

    N = len(siedco)
    lejanos = restas.index[(restas['distancia']>dist_max) | (restas['turno_lineal']>dist_t_max)]
    P = iniciarP(N, restas, lejanos, uniforme=True)
    restas = restas[['LONGITUD_X', 'LATITUD_Y', 'turno_lineal']]
    conv = [2]
    rep = []
    trans = []

    print('Inicio entrenamiento')
    
    while (conv[-1]>convergencia) & (len(conv)<=50):
        # Muestreo eventos de trasfondo
        # Retorna una matriz con un 1 en cada columna indicando, en la fila, el evento que lo causo
        # si i=j el evento es de trasfondo. Si i!=j, el evento i causo j.
        muestreo = np.vstack(tuple(np.random.multinomial(1, P[:,k], size=1) for k in range(N)))
        trasfondo = siedco.iloc[(np.where(np.diag(muestreo)==1))[0]]

        #Separar datos de trasfondo y replicas
        #Trasfondo:
        np.fill_diagonal(muestreo, 0)
        replicas = np.where(muestreo==1)
        densidadG = pd.DataFrame([0 for i in restas.index], index= restas.index)
        replicas = [(replicas[0][i],replicas[1][i]) for i in range(len(replicas[0]))]
        replicas=restas.loc[replicas]
        rep.append(len(replicas))
        print('Replicas: ' + str(len(replicas)/len(siedco)))
        
        #if len(replicas[0])>3:
            #entrena el kernel g con los datos de réplicas
        #    replicas = [(replicas[0][i],replicas[1][i]) for i in range(len(replicas[0]))]
        #    replicas=restas.loc[replicas]
        #    kdeG = KDEMultivariate(replicas, bw='cv_ml', var_type='ccu')
        #    rep.append(len(replicas))

        #else:
        #    replicas = pd.DataFrame(np.zeros((4,3)),index=restas.index[0:4])
        #    kdeG = KDEMultivariate(replicas, bw=[1,1,1], var_type='cco')
        #    rep.append(0)

        #densidadG[~densidadG.index.isin(lejanos)] = pd.DataFrame(kdeG.pdf(restas[~densidadG.index.isin(lejanos)]), index = densidadG[~densidadG.index.isin(lejanos)].index)

        #if len(trasfondo)>0:
            #entrena el kernel mu (espacial) con los datos de trasfondo
        #    kdeMu = KDEMultivariate(trasfondo[['LONGITUD_X','LATITUD_Y']], bw='cv_ml', var_type='cc')

            #entrena el kernel nu (temporal ciclico) con los datos de trasfondo
        #    kdeNu = KDEMultivariate(trasfondo[['turno_circular']], bw='cv_ml', var_type='u')

        #    trans.append(len(trasfondo))

        #else:
        #    trasfondo = pd.DataFrame(np.zeros((3,4)))
        #    trasfondo.columns = siedco.columns
        #    kdeMu = KDEMultivariate(trasfondo[['LONGITUD_X','LATITUD_Y']], bw=[1,1], var_type='cc')
        #    kdeNu = KDEMultivariate(trasfondo[['turno_circular']], bw=[1], var_type='o')
        #    trans.append(0)

        #densidadMu = kdeMu.pdf(siedco[['LONGITUD_X','LATITUD_Y']])
        #densidadNu = kdeNu.pdf(siedco[['turno_circular']])
        
        try:
            densidadG = pd.DataFrame([0 for i in restas.index], index= restas.index)
            densidadG[~restas.index.isin(lejanos)] = restas[~restas.index.isin(lejanos)].apply(kernel_g, axis = 1, args = (replicas, N, k_g))
            
            densidadMu = siedco.apply(kernel_mu, axis = 1, args = (siedco, N, k_mu))
    
            densidadNu = pd.Series(np.ones(len(siedco)))
        
            lam = np.concatenate([densidadNu[x]*densidadMu[x] + sum(densidadG.loc[x].values) for x in range(1,N)])

            lam = np.insert(lam, 0, densidadNu[0]*densidadMu[0])

            Pvieja = P

            #Actualizacion P

            P = np.hstack(list(map(lambda x: np.concatenate((np.array(densidadG.loc[[(x,i) for i in range(x)]]/lam[x]),
                                                             np.array([0 for i in range(N-x)]).reshape((N-x,1)))), range(N))))
            noCeros = [i for i in range(len(lam)) if lam[i]!=0]
            P[range(N), range(N)] = 0
            P[[i for i in range(N) if i in noCeros],[i for i in range(N) if i in noCeros]] = [densidadMu[j]*densidadNu[j]/lam[j] for j in range(N) if j in noCeros]

            conv.append(np.linalg.norm(Pvieja - P)/np.linalg.norm(Pvieja))

            #print("Termino iteracion localidad " + localidad + ' ' + str(time.time()) + '. Convergencia: ' + str(conv[-1])  )
            print("Termino iteracion " + str(time.time()) + '. Convergencia: ' + str(conv[-1])  )
        
        except:
            print('Reinicia entrenamiento')
            restas = matriz_distancias(siedco[['LONGITUD_X', 'LATITUD_Y', 'turno_lineal']])
            P = iniciarP(N, restas, lejanos)
            restas = restas[['LONGITUD_X', 'LATITUD_Y', 'turno_lineal']]
            if k_g-10 >= 50:
                k_g = k_g-10
            else:
                conv.append(0)            
        
    #graficar_resultados(conv[1::], directorio+'/plots/'+'_'.join(['conv', localidad, descriptor]), convergencia, kind = 'Convergencia')
    #graficar_resultados(rep, directorio+'/plots/'+'_'.join(['rep', localidad, descriptor]), kind = '# Crímenes Réplica')
    #graficar_resultados(trans, directorio+'/plots/'+'_'.join(['trans', localidad, descriptor]), kind = '# Crímenes Transfondo')

    tiempo = ((time.time() - inicio)/60)

    #return {'g': kdeG, 'mu': kdeMu, 'nu': kdeNu, 'datos': siedco, 'replicas': replicas, 'cod_localidad':localidad, 'tiempo_entrenamiento':tiempo, 'convergencia':conv[-1]}
    #return {'datos': siedco, 'replicas': replicas, 'cod_localidad':localidad, 'tiempo_entrenamiento':tiempo, 'convergencia':conv[-1]}
    return {'datos': siedco, 'replicas': replicas, 'tiempo_entrenamiento':tiempo, 'convergencia':conv[-1]}

def haversine(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371):
    """Funcion para obtener la distancia Haversine de dos puntos. Tiene en cuenta la curvatura de la tierra.
    slightly modified version: of http://stackoverflow.com/a/29546836/2901002

    lat1: Latitud del primer punto
    lon1: Longitud del primer punto
    lat2: Latitud del segundo punto
    lon2: Longitud del segundo punto
    to_radians: Parametro que indica si las coordenadas deben ser convertidas a radianes
    earth_radius: Radio de la tierra
        
    Retorna:
    Distancia Haversine entre dos puntos especificados
    """
    if to_radians:
        lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    a = np.sin((lat2-lat1)/2.0)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2

    return pd.Series(earth_radius * 2 * np.arcsin(np.sqrt(a)))


def matriz_distancias(datos):
    """Construccion de matriz de distancias espacio-temporales de los crimenes
    
     datos: Datos historicos de crimenes

     Retorna:
     Matriz de distancias espaciales y temporales de los crimenes ocurridos en la base de datos

    """
    N = len(datos)
    new = list(map(lambda x: datos.iloc[(N-x):].reset_index(drop = True)-datos.iloc[:x], [i for i in reversed(range(1,N))]))
    new = pd.concat(new)
    new = new.assign(indice0 = pd.concat(list(map(lambda x: pd.Series([i for i in range(N-1-x)]), [i for i in range(N-1)]))))
    new = new.assign(indice1 =pd.concat(list(map(lambda x: pd.Series([i for i in range(x,N)]), [i for i in range(1,N)])) ))
    new = new.assign(distancia = pd.concat(list(map(lambda x: haversine(datos['LATITUD_Y'].iloc[(N-x):].reset_index(drop = True), datos['LONGITUD_X'].iloc[(N-x):].reset_index(drop = True), datos['LATITUD_Y'].iloc[:x].reset_index(drop = True), datos['LONGITUD_X'].iloc[:x].reset_index(drop = True)) , [i for i in reversed(range(N))]))))
    new = new.set_index(['indice1', 'indice0'], drop=True)
    return new

def iniciarP(N, restas, lejanos, uniforme = False):
    """Inicializacion de la matriz P
    
     N: Numero de eventos en la base de datos
     restas: Matriz de distancias espacio-temporales de los crimenes
     lejanos: Indices de los eventos que no pueden ser considerados replica por superar maxima distancia espacial o temporal permitidas
     uniforma: Parametro que indica como se inicializa la matriz, si uniforme = True se inicializa con una distribucion uniforme, de lo contrario con normal bivariada en espacio y exponencial en tiempo

     Retorna:
     Matriz P inicial de probabilidades de ser crimen de transfondo o replica

    """
    if uniforme:
        P = np.transpose(np.vstack((np.concatenate(([1], [0 for j in range(N-1)])),tuple(np.concatenate((np.array([0.5/(i-1) for j in range(i-1)]),[0.5], [0 for j in range(N-i)])) for i in range(2,N+1)))))
    else:
        temp = pd.DataFrame(np.exp(-restas['turno_lineal']/14)*np.exp(-(restas['distancia']**2)/(2*0.01**2)))
        temp.reset_index(inplace=True)
        temp = temp.sort_values(['indice1', 'indice0'])
        P = np.transpose(np.vstack((np.concatenate(([1], [0 for j in range(N-1)])), tuple(np.concatenate((np.array(temp[temp['indice1']==i][0]), [1], [0 for j in range(N-i-1)])) for i in range(1, N)))))
    P[[b for (a,b) in lejanos],[a for (a,b) in lejanos]] = 0
    P = P/P.sum(axis=0)
    return P

def kernel_nu(punto, datos, N, k=100):

    temp = datos/datos.std()
    neighbors = NearestNeighbors(n_neighbors=k).fit(temp)
    try:
        vecinos = neighbors.kneighbors(temp)[0][:,-1]
    except:
        neighbors = NearestNeighbors(n_neighbors=len(temp)).fit(temp)
        vecinos = neighbors.kneighbors(temp)[0][:,-1]
    
    vecinos[np.where(vecinos==0)[0]]=min(vecinos[np.where(vecinos>0)[0]])
    vecinos = pd.DataFrame(vecinos)
    vecinos.columns = datos.columns
    
    nu = (punto - datos)**2
    nu.reset_index(drop=True, inplace=True)
    nu = nu/(2*datos.std()**2*vecinos**2)
    nu = np.exp(-nu)
    nu = nu/(datos.std()*(2*np.pi)**(1/2)*vecinos)
    return nu['turno_circular'].sum()/N


def kernel_mu(punto, datos, N, k=15):

    datos = datos[datos['turno_circular'] == punto['turno_circular']]
    datos = datos[['LONGITUD_X', 'LATITUD_Y']]
    punto = punto[['LONGITUD_X', 'LATITUD_Y']]
    temp = datos/datos.std()
    neighbors = NearestNeighbors(n_neighbors=k).fit(temp)
    try:
        vecinos = neighbors.kneighbors(temp)[0][:,-1]
    except:
        neighbors = NearestNeighbors(n_neighbors=len(temp)).fit(temp)
        vecinos = neighbors.kneighbors(temp)[0][:,-1]
    
    vecinos[np.where(vecinos==0)[0]] = min(vecinos[np.where(vecinos>0)[0]])
    vecinos = pd.DataFrame({'LONGITUD_X': vecinos, 'LATITUD_Y': vecinos})
    
    mu = (punto - datos)**2
    mu.reset_index(drop=True, inplace=True)
    mu = mu/(2*datos.std()**2*vecinos**2)
    mu = mu.sum(axis=1)
    mu = np.exp(-mu)
    mu = mu/(np.prod(datos.std())*(2*np.pi)*vecinos['LONGITUD_X']**2)
    return mu.mean()


def kernel_g(punto, datos, N, k=15):

    temp = datos/datos.std()
    neighbors = NearestNeighbors(n_neighbors=k).fit(temp)
    try:
        vecinos = neighbors.kneighbors(temp)[0][:,-1]
    except:
        neighbors = NearestNeighbors(n_neighbors=len(temp)).fit(temp)
        vecinos = neighbors.kneighbors(temp)[0][:,-1]
    
    vecinos[np.where(vecinos==0)[0]] = min(vecinos[np.where(vecinos>0)[0]])
    vecinos = pd.DataFrame({'LONGITUD_X': vecinos, 'LATITUD_Y': vecinos, 'turno_lineal':vecinos})
    
    g = (punto - datos)**2
    g.reset_index(drop=True, inplace=True)
    g = g/(2*datos.std()**2*vecinos**2)
    g = g.sum(axis=1)
    g = np.exp(-g)
    g = g/(np.prod(datos.std())*(2*np.pi)**(3/2)*vecinos['LONGITUD_X']**3)

    return g.sum()/N


def temperatura_ventana(punto, datos_entrenamiento, replicas, dist_max=0.3):
    """Retorna la intensidad esperada de crimen para un punto geografico y turno especificado, en la ventana de evaluacion 

     Parametros:
     lon: Longitud del punto a evaluar
     lat: Latitud del punto a evaluar
     ventana_linal: Ventana de evaluacion del punto
     turno_circular: Turno para la evaluacion
     localidad: Localidad a la que pertenece el punto y para la cual se debe cargar la densidad respectiva
     descriptor: Version del modelo deseada
     dist_max: Distancia maxima entre crimenes considerados replica

     Retorna:
     valor promedio en la ventana de evaluacion de la intensidad en el punto geográfico y turno especificados.

    """
    try:
        #grilla = pd.DataFrame({'LONGITUD_X': lon, 'LATITUD_Y':lat,
        #                       'turno_circular':[turno_circular]})
        grilla = pd.DataFrame(punto).T
        grilla.reset_index(drop=True, inplace=True)
        #densidades = cargar_densidades(localidad, descriptor)

        #nu = densidades['nu']
        #mu = densidades['mu']
        #g = densidades['g']
        #datos_entrenamiento = densidades['datos']

        datos_entrenamiento['lon_rad'] = np.radians(datos_entrenamiento['LONGITUD_X'])
        datos_entrenamiento['lat_rad'] = np.radians(datos_entrenamiento['LATITUD_Y'])
        grilla['lon_rad'] = np.radians(grilla['LONGITUD_X'])
        grilla['lat_rad'] = np.radians(grilla['LATITUD_Y'])

        distancia = 6371*DistanceMetric.get_metric('haversine').pairwise(grilla[['lon_rad', 'lat_rad']], datos_entrenamiento[['lon_rad', 'lat_rad']])
        distancia = distancia.min(axis=0)

        #datos_entrenamiento = datos_entrenamiento[distancia<dist_max].reset_index(drop=True)

        #densidadMu = mu.pdf(grilla[['LONGITUD_X', 'LATITUD_Y']])
        #densidadNu = nu.pdf(grilla[['turno_circular']])

        #intensidades = list()


        #for time in ventana_lineal:
        #    grilla['turno_lineal'] = time

        #    g_temp = grilla.loc[0,['LONGITUD_X', 'LATITUD_Y', 'turno_lineal']]
        #    datos_entrenamiento_temp = datos_entrenamiento.loc[:,['LONGITUD_X', 'LATITUD_Y', 'turno_lineal']]
        #    resta = g_temp - datos_entrenamiento_temp

        #    densidadG = np.sum(g.pdf(resta))

        #    intensidades.append(densidadMu*densidadNu + densidadG)


        #return np.mean(intensidades)
        
        k_g = 300
        k_mu = 5
        
        N = len(datos_entrenamiento)
        densidadMu = kernel_mu(grilla.loc[0], datos_entrenamiento, N, k_mu)        
        if min(distancia)>=dist_max:
            return(densidadMu/3)
        else:
            datos_entrenamiento = datos_entrenamiento[distancia<dist_max].reset_index(drop=True)
            #replicas = densidades['replicas']
            
            g_temp = grilla.loc[0, ['LONGITUD_X', 'LATITUD_Y', 'turno_lineal']]-datos_entrenamiento[['LONGITUD_X', 'LATITUD_Y', 'turno_lineal']]
            g_temp = g_temp.apply(kernel_g, axis = 1, args = (replicas, N, k_g))
            densidadG = np.sum(g_temp)        
            intensidad = densidadMu + densidadG
        return(intensidad)
        
    except:
        #print('Error: ' + str(lat) + ', ' + str(lon) + ', ' + str(turno_circular))
        print('Error')
        return(0)
        


In [ ]:
data=pd.read_csv('C:/Users/jsmor/Dropbox (Quantil)/Crimen/AddressBias/mohler_santafe.csv')
data.drop(columns=['Unnamed: 0'], inplace=True)
data.FECHA_HECHO=pd.to_datetime(data.FECHA_HECHO)
#data=data.loc[data.Localidad!=99,:]
data.columns=['Date', 'hour', 'long', 'lat', 'weekday', 'localidad', 'Neighborhood','knowledge', 'type', 'homicides', 'reports']
data.sort_values('Date', inplace=True)

In [ ]:
data['turno_lineal'] = pd.Categorical(data['Date']).codes
data['turno_circular'] = data['weekday']%7
data.rename(columns = {'long':'LONGITUD_X', 'lat':'LATITUD_Y'}, inplace = True)

In [ ]:
data.head()

In [ ]:
train_periods = 180
desfase = 0
data_train = data[['LONGITUD_X', 'LATITUD_Y', 'turno_circular', 'turno_lineal']]
data_train = data_train[(data_train['turno_lineal']<desfase+train_periods) &
                       (data_train['turno_lineal']>=desfase)]
data_train.reset_index(drop=True, inplace=True)

In [ ]:
print(len(data_train))
data_train.head()

In [8]:
%%time
mohler_model = entrenar(data_train, convergencia=0.03, dist_max=0.2, dist_t_max=60)

Inicio entrenamiento
Replicas: 0.11243144424131626
Termino iteracion 1558885117.960617. Convergencia: 0.5148833244701592
Replicas: 0.3153564899451554
Termino iteracion 1558885521.743445. Convergencia: 0.34629939143457467
Replicas: 0.49725776965265084
Termino iteracion 1558886094.8815982. Convergencia: 0.26320932114055984
Replicas: 0.593235831809872
Termino iteracion 1558886713.7129319. Convergencia: 0.17677069277053448
Replicas: 0.6691042047531993
Termino iteracion 1558887358.47455. Convergencia: 0.16197404395350404
Replicas: 0.6791590493601463
Termino iteracion 1558888002.0710368. Convergencia: 0.09010926408105151
Replicas: 0.680073126142596
Termino iteracion 1558895124.0151212. Convergencia: 0.06745666931486363
Replicas: 0.6681901279707495
Termino iteracion 1558895504.7383397. Convergencia: 0.05869823407203936
Replicas: 0.6590493601462523
Termino iteracion 1558895846.7899349. Convergencia: 0.06862266059720966
Replicas: 0.6535648994515539
Termino iteracion 1558896166.8373768. Converge

In [9]:
from shapely.geometry import Point
import geopandas as gpd
import numpy as np
import pandas as pd
from sklearn.neighbors import DistanceMetric
from scipy import stats
from shapely.ops import cascaded_union

In [10]:
santafe = gpd.read_file('C:/Users/jsmor/Dropbox (Quantil)/Isocronas (1)/upz/upz.shp')
santafe = santafe[(santafe['NOMBRE'] == 'SAGRADO CORAZON') | (santafe['NOMBRE'] == 'LA MACARENA') | (santafe['NOMBRE'] == 'LAS NIEVES')]
santafe.to_crs(epsg=4326, inplace=True)
poligono = gpd.GeoSeries(cascaded_union(santafe['geometry'])).iloc[0]

In [11]:
def assing_upz(x, y, shape):
    punto = Point(x, y)
    barrios = ['SAGRADO CORAZON', 'LA MACARENA', 'LAS NIEVES']
    for upz in barrios:
        if punto.within(shape[shape['NOMBRE']==upz]['geometry'].iloc[0]):
            return upz
        
def crear_grilla(poligono, delta=0.0031):

    puntos_lon = []
    puntos_lat = []
    #num = (dar_area(poligono)/10**6)*densidad
    min_x, min_y, max_x, max_y = poligono.bounds
    #space_x = abs(max_x-min_x)/math.sqrt(num)
    #space_y = abs(max_y-min_y)/math.sqrt(num)
    x = min_x
    y = min_y
    while x <= max_x:
        while y <= max_y:
            random_point = Point(x,y)
            if (random_point.within(poligono)):
                puntos_lon.append(x)
                puntos_lat.append(y)
            y = y + delta
        y = min_y
        x = x + delta
    
    grilla = pd.DataFrame({'LONGITUD_X':puntos_lon, 'LATITUD_Y':puntos_lat})
        
    return grilla

In [12]:
data_latice = crear_grilla(poligono)

In [13]:
t = desfase + train_periods
data_latice['turno_lineal'] = t
data_latice['turno_circular'] = t%7

In [14]:
data_latice

,LONGITUD_X,LATITUD_Y,turno_lineal,turno_circular
0,-74.081608,4.598156,180,5
1,-74.078508,4.601256,180,5
2,-74.078508,4.604356,180,5
3,-74.075408,4.604356,180,5
4,-74.075408,4.607456,180,5
5,-74.072308,4.601256,180,5
6,-74.072308,4.604356,180,5
7,-74.072308,4.607456,180,5
8,-74.072308,4.610556,180,5
9,-74.072308,4.613656,180,5


In [15]:
%%time
data_latice['intensidad'] = data_latice.apply(temperatura_ventana, axis = 1,
                                             args = (mohler_model['datos'], mohler_model['replicas']))

Wall time: 4min 21s


In [16]:
data_latice['upz'] = [assign_upz(data_latice['LONGITUD_X'].iloc[i], data_latice['LATITUD_Y'].iloc[i], 
                                 santafe) for i in data_lattice.index]

NameError: name 'data_lattice' is not defined

In [ ]:
data_latice.groupby('upz')['intensidad'].mean().reset_index()